# Hate Speech Detector 2.0
---
**Hateful and vulgar phrases dictionary extension**
1. Hateful and vulgar phrases analysis:
    1. Loading set of raw hateful and vulgar phrases (sanitized tweets)
    2. **Tagged** and **lemmatized phrases**
    3. Set of **wordnet synonymic** hateful phrases
    4. Set of **word-vector similar** hateful phrases
    5. Distinctive combination of three above sets
2. Saving of each of extended dictionaries and visualizations.

In [1]:
import numpy as np
import pandas as pd

import itertools
from scipy.spatial.distance import cosine

from polyglot.text import Text
from polyglot.downloader import downloader

from combo.predict import COMBO
import spacy
from pyplwnxml import PlwnxmlParser

from tqdm.notebook import tqdm

import os
import csv
import pickle
import string

In [ ]:
LABELS_SMALL = ['wyz', 'groz', 'wyk', 'odcz', 'pon', 'styg', 'szan']
LABELS_V_SMALL = LABELS_SMALL + ['vulg']

HATEFUL_RAW_DIR = 'data/hateful/raw_{}.txt'
HATEFUL_LEMM_DIR = 'data/hateful/lemm_{}.txt'
HATEFUL_EXT_DIR = 'data/hateful/ext_{}.txt'
VULGARS_RAW_DIR = 'data/vulgars/raw_{}.txt'
VULGARS_LEMM_DIR = 'data/vulgars/lemm_{}.txt'
VULGARS_EXT_DIR = 'data/vulgars/ext_{}.txt'

WORDNET_PATH = 'models/plwordnet_3_0/plwordnet-3.0.xml'
TAGGER_MODEL = 'polish-herbert-base'
SPACY_PL_MODEL = 'pl_model'

In [2]:
pd.set_option('display.max_colwidth', 400)

In [3]:
with open('data/other/polish_stopwords.txt', 'r') as f:\
    polish_stopwords = f.read().split('\n')[:-1]
polish_stopwords[:10]

['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ']

In [4]:
from polyglot.detect.base import logger as polyglot_logger
polyglot_logger.setLevel("ERROR")

## Hateful and vulgar phrases analysis

### Loading raw phrases

In [ ]:
def load_raw_phrases():
    aphr = list([])
    for label in LABELS_SMALL:
        with open(HATEFUL_RAW_DIR.replace('{}', label), 'r') as f:
            aphr.append(np.array(f.read().split(';')))
    with open(VULGARS_RAW_DIR.replace('{}', LABELS_V_SMALL[-1]), 'r') as f:
        aphr.append(np.array(f.read().split(';')))
    
    return np.array(aphr)

all_phr = load_raw_phrases()

In [5]:
phr_wyz = all_phr[0]
phr_wyz[:5]

array(['komuchów', ' gonić komuchów', 'Kwaśniewski [...] idiota',
       'pojebało kogoś', 'politycy [...] kłapania dziobem'], dtype='<U75')

In [6]:
phr_groz = all_phr[1]
phr_groz[:5]

array(['sądy [...] skasują', 'bomb[...] domy podejrzanych',
       'policzek [...] nie zaszkodzi', 'że was zajebiemy',
       'sądy gówniane'], dtype='<U75')

In [7]:
phr_wyk = all_phr[2]
phr_wyk[:5]

array(['ryj [...] zakazany', 'prezydent nie wart szacunku',
       ' Premiar morderca, zdrajca', 'wypierdoliliśmy ją z roboty',
       ' nie ma kasy na sądy [...] nam nie skoczy'], dtype='<U73')

In [8]:
phr_odcz = all_phr[3]
phr_odcz[:5]

array(['komisja śledcza z pachołkami bez wiedzy',
       'prezydent nie wart szacunku', ' Premiar morderca, zdrajca',
       'bomb[...] domy podejrzanych', 'esbeckie złogi'], dtype='<U72')

In [9]:
phr_pon = all_phr[4]
phr_pon[:5]

array(['odbierajmy [...] bałwanom', 'spocona świnia', ' świnia',
       'durne angielskie przepisy', 'Kwaśniewski [...] idiota'],
      dtype='<U75')

In [10]:
phr_styg = all_phr[5]
phr_styg[:5]

array(['sądy [...] to zbytek', ' #Józefmoneta',
       'PiS gwałci żeby nie robić aborcji', 'ryj [...] zakazany',
       'politycy [...] kłapania dziobem'], dtype='<U83')

In [11]:
phr_szan = all_phr[6]
phr_szan[:5]

array(['wypierdalaj stąd, bo zaliczysz zgon',
       ' jesteś kurwą, wypierdalaj', ' wypierdalaj',
       'jak nie odsuniecie [...] to zamkniemy was w pierdlu',
       'jak dalej będziesz tworzyć trudne sądy [...] przez Ciebie przejdę ząłamanie nerwowe'],
      dtype='<U109')

In [12]:
phr_vulg = all_phr[7]
phr_vulg[:5]

array(['CJK', 'ChWDP', 'JP', 'MILF', 'WTF'], dtype='<U40')

In [ ]:
del all_phr

### Lemmatized phrases
Get lemmas of each phrase token and combine them into one sentence.

In [13]:
tagger = COMBO.from_pretrained(TAGGER_MODEL)

In [14]:
def lemmatize_text(text):
    text = text.replace('#', '').replace('[...]', '')
    sentence = tagger(text)
    
    lemmas = [token.lemma.lower() for token in sentence.tokens if token.deprel != 'punct']
    lemm_text = ' '.join(lemmas)
    
    return lemm_text

def get_lemmatized_phrases(phrases, save_file=None):
    lemm_phrases = list([])
    
    for phrase in tqdm(phrases, leave=None):
        lemm_phrase = lemmatize_text(phrase)
        if lemm_phrase not in lemm_phrases:
            lemm_phrases.append(lemm_phrase)
    
    if save_file:
        with open(save_file, 'w') as f:
            f.write(';'.join(lemm_phrases))
    
    return np.array(lemm_phrases)

In [15]:
lemmatize_text('sądy cycki zrobiła ja pierdolę')

'sąd cycki zrobić ja pierdolać'

In [16]:
lemmphr_wyz = get_lemmatized_phrases(phr_wyz, save_file=HATEFUL_LEMM_DIR.replace('{}', LABELS_V_SMALL[0]))
lemmphr_wyz[:5]

array(['komuch', 'gonić komuch', 'kwaśniewski idiota', 'pojebać ktoś',
       'polityk kłapać dziób'], dtype='<U64')

In [17]:
lemmphr_groz = get_lemmatized_phrases(phr_groz, save_file=HATEFUL_LEMM_DIR.replace('{}', LABELS_V_SMALL[1]))
lemmphr_groz[:5]

array(['sąd skasywać', 'bomb dom podejrzany', 'policzek nie zaszkodzić',
       'że wy zajeb', 'sąd gówniany'], dtype='<U71')

In [18]:
lemmphr_wyk = get_lemmatized_phrases(phr_wyk, save_file=HATEFUL_LEMM_DIR.replace('{}', LABELS_V_SMALL[2]))
lemmphr_wyk[:5]

array(['ryj zakazać', 'prezydent nie wart szacunek',
       'premiar morderca zdrajca', 'wypierdolićić on z robota',
       'nie mieć kasa na sąd my nie skoczyć'], dtype='<U68')

In [19]:
lemmphr_odcz = get_lemmatized_phrases(phr_odcz, save_file=HATEFUL_LEMM_DIR.replace('{}', LABELS_V_SMALL[3]))
lemmphr_odcz[:5]

array(['komisja śledczy z pachołek bez wiedza',
       'prezydent nie wart szacunek', 'premiar morderca zdrajca',
       'bomb dom podejrzany', 'esbecki złoga'], dtype='<U68')

In [20]:
lemmphr_pon = get_lemmatized_phrases(phr_pon, save_file=HATEFUL_LEMM_DIR.replace('{}', LABELS_V_SMALL[4]))
lemmphr_pon[:5]

array(['odbierać bałwan', 'spocić świnia', 'świnia',
       'durny angielski przepis', 'kwaśniewski idiota'], dtype='<U68')

In [21]:
lemmphr_styg = get_lemmatized_phrases(phr_styg, save_file=HATEFUL_LEMM_DIR.replace('{}', LABELS_V_SMALL[5]))
lemmphr_styg[:5]

array(['sąd to zbytek', 'józefmoneta',
       'pis gwałcić żeby nie robić aborcja', 'ryj zakazać',
       'polityk kłapać dziób'], dtype='<U71')

In [22]:
lemmphr_szan = get_lemmatized_phrases(phr_szan, save_file=HATEFUL_LEMM_DIR.replace('{}', LABELS_V_SMALL[6]))
lemmphr_szan[:5]

array(['wypierdalać stąd bo zaliczyć zgon', 'być kurwa wypierdalać',
       'wypierdalać', 'jak nie odsunąe to zamknąć wy w pierdel',
       'jak daleko być tworzyć trudny sąd przez ty przejść ząłamanie nerwowy'],
      dtype='<U88')

In [23]:
lemmphr_vulg = get_lemmatized_phrases(phr_vulg, save_file=VULGARS_LEMM_DIR.replace('{}', LABELS_V_SMALL[7]))
lemmphr_vulg[:5]

array(['cjk', 'chwdp', 'jp', 'milf', 'wtf'], dtype='<U39')

### Wordnet synonymic
Get negative-sentiment polish-wordnet synonymic words for each lemmatized word and perform cartesian product over listed lists of word synonyms.

In [24]:
pl_wordnet = PlwnxmlParser(WORDNET_PATH).read_wordnet()

In [25]:
def synonymic_phrases(phrase, lemmatized=True, stopwords=[], negative_sentiment_only=True):
    synonymic_phrases_options = list([])
    p = phrase if lemmatized else lemmatize_text(phrase)
    
    lemm_words = list([w for w in p.split(' ') if w not in stopwords])
    
    for lemm_word in lemm_words:
        lemm_word_options = list([])
        for lemm in pl_wordnet.lemma(lemm_word):
            for synset in lemm.synsets:
                for lu in synset.lexical_units:
                    if (not negative_sentiment_only or lu.sentiment in ['- m', '- s'])\
                    and lu.name not in lemm_word_options:
                        lemm_word_options.append(lu.name)
        
        if len(lemm_word_options) == 0:
            lemm_word_options.append(lemm_word)
        synonymic_phrases_options.append(lemm_word_options)
    
    options = list(itertools.product(*synonymic_phrases_options))
    options = list([' '.join(option) for option in options])
    if p in options:
        options.remove(p)
    
    return options

def get_synonymic_phrases(phrases, lemmatized=True, stopwords=[], save_file=None):
    all_syn_phrases = list([])
    
    if save_file:
        with open(save_file, 'w') as f:
            pass
    
    t = tqdm(phrases, leave=None)
    for phrase in t:
        syn_phrases = synonymic_phrases(phrase, lemmatized=lemmatized, stopwords=stopwords)
        for sp in syn_phrases:
            if sp not in all_syn_phrases:
                all_syn_phrases.append(sp)
                
                if save_file:
                    with open(save_file, 'a') as f:
                        f.write(f'{sp};')
                
    t.set_postfix_str(f'Found so far: {len(all_syn_phrases)}')
    
    return np.array(all_syn_phrases)

In [26]:
synonymic_phrases('chory nierychliwy sąd iść precz z taki dupa', stopwords=polish_stopwords)

['chory nierychliwy sąd iść precz dupa']

In [27]:
synphr_wyz = get_synonymic_phrases(lemmphr_wyz, stopwords=polish_stopwords)
synphr_wyz[:5]

array(['pojebać', 'czuć', 'prezydent wart szacunek', 'chntyro',
       'chui dupa'], dtype='<U62')

In [28]:
synphr_groz = get_synonymic_phrases(lemmphr_groz, stopwords=polish_stopwords)
synphr_groz[:5]

array(['policzek zaszkodzić', 'zajeb', 'ruski wypieprzyć',
       'prezydent czarny dupa', 'wyczyścić sędzia'], dtype='<U64')

In [29]:
synphr_wyk = get_synonymic_phrases(lemmphr_wyk, stopwords=polish_stopwords)
synphr_wyk[:5]

array(['prezydent wart szacunek', 'wypierdolićić robota',
       'mieć kasa sąd skoczyć', 'ruski wypieprzyć', 'wyczyścić sędzia'],
      dtype='<U64')

In [30]:
synphr_odcz = get_synonymic_phrases(lemmphr_odcz, stopwords=polish_stopwords)
synphr_odcz[:5]

array(['komisja śledczy pachołek wiedza', 'prezydent wart szacunek',
       'esbecki złoga psl', 'sąd cycki zrobić pierdolać', 'chcieć menda'],
      dtype='<U64')

In [31]:
synphr_pon = get_synonymic_phrases(lemmphr_pon, stopwords=polish_stopwords)
synphr_pon[:5]

array(['sąd zbytek', 'pis gwałcić robić aborcja',
       'komisja śledczy pachołek wiedza', 'czuć',
       'prezydent wart szacunek'], dtype='<U64')

In [32]:
synphr_styg = get_synonymic_phrases(lemmphr_styg, stopwords=polish_stopwords)
synphr_styg[:5]

array(['sąd zbytek', 'pis gwałcić robić aborcja',
       'prezydent wart szacunek', 'sąd patologia pierdolić',
       'jews kraść milion'], dtype='<U62')

In [33]:
synphr_szan = get_synonymic_phrases(lemmphr_szan, stopwords=polish_stopwords)
synphr_szan[:5]

array(['wypierdalać stąd zaliczyć zgon', 'kurwa wypierdalać',
       'odsunąe zamknąć pierdel',
       'tworzyć trudny sąd przejść ząłamanie nerwowy',
       'akta stół morda kubełl'], dtype='<U67')

In [34]:
synphr_vulg = get_synonymic_phrases(lemmphr_vulg, stopwords=polish_stopwords)
synphr_vulg[:5]

array(['gówno', 'bać własny dupa', 'brandzlować', 'certolić',
       'chuj bombka strzelić choinka'], dtype='<U31')

### Word-vector similar phrases
Get negative-sentiment word-vector similar words for each lemmatized word and perform cartesian product over listed lists of word synonyms.

In [35]:
pl_nlp = spacy.load(SPACY_PL_MODEL)

/home/jakub/Dokumenty/venv36/lib/python3.6/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'pl_model' (1.0.0) requires spaCy v2.1,<3 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [70]:
def similar_phrases(phrase, sim_threshold=0.6, lemmatized=True, stopwords=[], negative_sentiment_only=True):
    synonymic_phrases_options = list([])
    p = phrase if lemmatized else lemmatize_text(phrase)

    # detect and delete invalid characters first
    invalid = set()
    for i, ch in enumerate(p):
        try:
            Text(f"Char: {ch}").words
        except:
            invalid.add(ch)
    for ch in invalid:
        p = p.replace(ch, '')

    wordtokens = list([wt for wt in pl_nlp(str(p)) if wt.text not in stopwords and wt.has_vector])
    cosine_similarity = lambda x, y: 1 - cosine(x, y)

    similar_phrases_options = list([])
    for wordtoken in tqdm(wordtokens, leave=None):
        wordtoken_options = list([])
        words = list([word for word in pl_nlp.vocab if word.has_vector and '.' not in word.text])
        for word in tqdm(words, leave=None):
            try:
                sentiment = Text(word.text).words[0].polarity
            except Exception:
                sentiment = 0

            lemma = tagger(word.text).tokens[0].lemma.lower()
            sim = cosine_similarity(wordtoken.vector, word.vector)

            if (not negative_sentiment_only or sentiment == -1) \
                    and sim >= sim_threshold and lemma not in wordtoken_options:
                wordtoken_options.append(lemma)

        if len(wordtoken_options) == 0:
            wordtoken_options.append(wordtoken.text)
        similar_phrases_options.append(wordtoken_options)

    options = list(itertools.product(*similar_phrases_options))
    options = list([' '.join(option) for option in options])
    if p in options:
        options.remove(p)

    return options

def get_similar_phrases(phrases, lemmatized=True, stopwords=[], save_file=None):
    all_sim_phrases = list([])
    
    if save_file:
        with open(save_file, 'w') as f:
            pass
    
    t = tqdm(phrases, leave=None)
    for phrase in t:
        sim_phrases = similar_phrases(phrase, lemmatized=lemmatized, stopwords=stopwords)
        for sp in sim_phrases:
            if sp not in all_sim_phrases:
                all_sim_phrases.append(sp)
                
                if save_file:
                    with open(save_file, 'a') as f:
                        f.write(f'{sp};')
        
        t.set_postfix_str(f'Found so far: {len(all_sim_phrases)}')
    
    return np.array(all_sim_phrases)

In [37]:
similar_phrases('chory nierychliwy sąd iść precz z taki dupa', stopwords=polish_stopwords)

['chory nierychliwy sąd iść precz dupa']

In [38]:
simphr_wyz = get_similar_phrases(lemmphr_wyz, stopwords=polish_stopwords)
simphr_wyz[:5]

array(['pojebać', 'czuć', 'prezydent wart szacunek', 'chntyro',
       'chui dupa'], dtype='<U62')

In [39]:
simphr_groz = get_similar_phrases(lemmphr_groz, stopwords=polish_stopwords)
simphr_groz[:5]

array(['policzek zaszkodzić', 'zajeb', 'ruski wypieprzyć',
       'prezydent czarny dupa', 'wyczyścić sędzia'], dtype='<U64')

In [40]:
simphr_wyk = get_similar_phrases(lemmphr_wyk, stopwords=polish_stopwords)
simphr_wyk[:5]

array(['prezydent wart szacunek', 'premiar kryminalista zdrajca',
       'premiar gwałciciel zdrajca', 'premiar przestępca zdrajca',
       'wypierdolićić robota'], dtype='<U64')

In [41]:
simphr_odcz = get_similar_phrases(lemmphr_odcz, stopwords=polish_stopwords)
simphr_odcz[:5]

array(['komisja śledczy pachołek wiedza', 'prezydent wart szacunek',
       'premiar gwałciciel zdrajca', 'premiar przestępca zdrajca',
       'premiar kryminalista zdrajca'], dtype='<U61')

In [42]:
simphr_pon = get_similar_phrases(lemmphr_pon, stopwords=polish_stopwords)
simphr_pon[:5]

array(['głupi angielski przepis', 'bezczelny angielski przepis',
       'kwaśniewski głupi', 'sąd zbytek', 'pis mordować robić aborcja'],
      dtype='<U62')

In [72]:
simphr_styg = get_similar_phrases(lemmphr_styg, stopwords=polish_stopwords)
simphr_styg[:5]

array(['sąd zbytek', 'pis mordować robić aborcja',
       'pis zabijać robić aborcja', 'prezydent wart szacunek',
       'premiar zabójca zdrajca'], dtype='<U28')

In [71]:
simphr_szan = get_similar_phrases(lemmphr_szan, stopwords=polish_stopwords)
simphr_szan[:5]

array(['wypierdalać stąd zaliczyć zgon', 'kurwa wypierdalać',
       'odsunąe wyrzucić pierdel',
       'niszczyć trudny sąd przejść ząłamanie nerwowy',
       'niszczyć ciężki sąd przejść ząłamanie nerwowy'], dtype='<U67')

In [74]:
simphr_vulg = get_similar_phrases(lemmphr_vulg, stopwords=polish_stopwords)
simphr_vulg[:5]

array(['gówno', 'bać własny dupa', 'brandzlować', 'certolić',
       'chuj bombka strzelić choinka'], dtype='<U28')

### Phrases combining
Combine appropriate lemmatized, word-vector similar and wordnet synonymic phrases and save into .txt file.

In [75]:
def combine_phrases(lemmatized, similar, synonymic, save_file=None):
    all_phrases = np.union1d(similar, synonymic)
    all_phrases = np.union1d(lemmatized, all_phrases)
    all_phrases = all_phrases[all_phrases != '']
    
    if save_file:
        with open(save_file, 'w') as f:
            f.write(';'.join(all_phrases))
    
    return all_phrases

In [76]:
allphr_wyz = combine_phrases(lemmphr_wyz, simphr_wyz, synphr_wyz)
allphr_wyz[:5]

array(['8 rok szabrowaliśt polska bezkarnie', '8 rok żerowaliśt',
       '8 rok żerowaliśt na my', 'a teraz won', 'adwokat diabeł'],
      dtype='<U64')

In [77]:
allphr_groz = combine_phrases(lemmphr_groz, simphr_groz, synphr_groz)
allphr_groz[:5]

array(['adamowicz drain the swamp', 'akta na stół albo morda w kubełl',
       'akta stół morda kubełl', 'bandyta zlikwidować sąd',
       'bez granat trudno szukać sprawiedliwość'], dtype='<U71')

In [78]:
allphr_wyk = combine_phrases(lemmphr_wyk, simphr_wyk, synphr_wyk)
allphr_wyk[:5]

array(['8 rok żerowaliśt', '8 rok żerowaliśt na my', 'a teraz won',
       'antypolski sąd precz polska', 'antypolski sąd precz z polska'],
      dtype='<U68')

In [79]:
allphr_odcz = combine_phrases(lemmphr_odcz, simphr_odcz, synphr_odcz)
allphr_odcz[:5]

array(['8 rok żerowaliśt', '8 rok żerowaliśt na my', 'a teraz won',
       'baran mieć wyrok', 'bomb dom podejrzany'], dtype='<U68')

In [80]:
allphr_pon = combine_phrases(lemmphr_pon, simphr_pon, synphr_pon)
allphr_pon[:5]

array(['8 rok człowiek mieć dupa', '8 rok człowiek mieć w dupa',
       '8 rok przekupić media sąd służba',
       '8 rok przekupić tylko media sąd służba', '8 rok żerowaliśt'],
      dtype='<U68')

In [81]:
allphr_styg = combine_phrases(lemmphr_styg, simphr_styg, synphr_styg)
allphr_styg[:5]

array(['8 rok człowiek mieć dupa', '8 rok człowiek mieć w dupa',
       '8 rok przekupić media sąd służba',
       '8 rok przekupić tylko media sąd służba',
       '8 rok szabrowaliśt polska bezkarnie'], dtype='<U71')

In [82]:
allphr_szan = combine_phrases(lemmphr_szan, simphr_szan, synphr_szan)
allphr_szan[:5]

array(['akta na stół albo morda w kubełl', 'akta stół morda kubełl',
       'być kurwa wypierdalać',
       'jak daleko być tworzyć trudny sąd przez ty przejść ząłamanie nerwowy',
       'jak nie odsunąe to zamknąć wy w pierdel'], dtype='<U88')

In [83]:
allphr_vulg = combine_phrases(lemmphr_vulg, simphr_vulg, synphr_vulg)
allphr_vulg[:5]

array(['a gówno', 'afa', 'bać się o własny dupa', 'bać własny dupa',
       'blad'], dtype='<U39')